

---



---


Training Code

---



---



In [1]:
import sys
system_directory = "/content/drive/MyDrive/Upwork/CellularImage-Unet/CodeSpace"
sys.path.append(system_directory)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
!pip install torch torchvision matplotlib tqdm numpy torchsummary torchinfo einops


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 68.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 99.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitl

In [4]:
from typing import Dict, Tuple
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from IPython.display import HTML
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from IPython.display import HTML
from tqdm import tqdm
import os

from typing import Dict, Tuple
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from IPython.display import HTML
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary
from torch.utils.data import DataLoader
from torchvision import models, transforms
from torchvision.utils import save_image, make_grid
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
import numpy as np
from IPython.display import HTML
# from diffusion_utilities import *
from tqdm import tqdm
import os
from torchinfo import summary


import torch
import torch.nn as nn
from einops import rearrange
from torch import einsum

import torch
import torch.nn as nn
from einops import rearrange
from torch import einsum


In [5]:
# Necessary imports
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange
from torch import einsum
import math

"""
Positional Encoding
"""
class PositionalEncoding2D(nn.Module):
    def __init__(self, channels):
        super(PositionalEncoding2D, self).__init__()
        if channels % 4 != 0:
            raise ValueError("Channels must be divisible by 4")
        self.channels = channels

    def forward(self, x):
        batch_size, channels, height, width = x.size()
        device = x.device

        num_pos_feats = channels // 2

        # Create grid of positions
        y_embed = torch.linspace(-1, 1, height, device=device).unsqueeze(1).repeat(1, width)  # (height, width)
        x_embed = torch.linspace(-1, 1, width, device=device).unsqueeze(0).repeat(height, 1)  # (height, width)

        # Calculate div_term
        dim_t = torch.arange(num_pos_feats // 2, dtype=torch.float32, device=device)
        dim_t = 10000 ** (2 * (dim_t // 2) / num_pos_feats)

        # Compute positional embeddings
        pos_x = x_embed.unsqueeze(-1) / dim_t
        pos_y = y_embed.unsqueeze(-1) / dim_t

        # Interleave sine and cosine for x and y
        pos_x = torch.stack((pos_x.sin(), pos_x.cos()), dim=-1).view(height, width, -1)
        pos_y = torch.stack((pos_y.sin(), pos_y.cos()), dim=-1).view(height, width, -1)

        # Concatenate x and y embeddings
        pos_emb = torch.cat((pos_y, pos_x), dim=-1)  # Shape: (height, width, channels)

        # Permute to match the shape (batch_size, channels, height, width)
        pos_emb = pos_emb.permute(2, 0, 1).unsqueeze(0).repeat(batch_size, 1, 1, 1)

        return pos_emb

"""
Model Utilities with Enhancements
"""
def get_num_groups(num_channels):
    for num_groups in [32, 16, 8, 4, 2, 1]:
        if num_channels % num_groups == 0:
            return num_groups
    return 1

class ResidualConvBlock(nn.Module):
    def __init__(self, in_channels, out_channels, is_res=True):
        super().__init__()

        self.same_channels = in_channels == out_channels
        self.is_res = is_res

        num_groups_in = get_num_groups(in_channels)
        num_groups_out = get_num_groups(out_channels)

        # Pre-activation residual block with GroupNorm
        self.norm1 = nn.GroupNorm(num_groups_in, in_channels)
        self.act1 = nn.GELU()
        self.conv1 = nn.Conv2d(in_channels, out_channels, 3, 1, 1)

        self.norm2 = nn.GroupNorm(num_groups_out, out_channels)
        self.act2 = nn.GELU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, 3, 1, 1)

        if not self.same_channels:
            self.conv1x1 = nn.Conv2d(in_channels, out_channels, 1)

    def forward(self, x):
        identity = x

        out = self.norm1(x)
        out = self.act1(out)
        out = self.conv1(out)

        out = self.norm2(out)
        out = self.act2(out)
        out = self.conv2(out)

        if self.is_res:
            if not self.same_channels:
                identity = self.conv1x1(identity)
            out += identity
            out = out / 1.414

        return out

class UnetDown(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UnetDown, self).__init__()

        self.conv = nn.Sequential(
            ResidualConvBlock(in_channels, out_channels),
            ResidualConvBlock(out_channels, out_channels),
        )
        self.down = nn.MaxPool2d(2)

    def forward(self, x):
        x = self.conv(x)
        x_down = self.down(x)
        return x_down, x  # Return both for skip connections

class UnetUp(nn.Module):
    def __init__(self, in_channels, skip_channels, out_channels):
        super(UnetUp, self).__init__()

        self.up = nn.Sequential(
            nn.Upsample(scale_factor=2, mode='nearest'),
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.GroupNorm(get_num_groups(out_channels), out_channels),
            nn.GELU(),
        )
        self.conv = nn.Sequential(
            ResidualConvBlock(out_channels + skip_channels, out_channels),
            ResidualConvBlock(out_channels, out_channels),
        )

    def forward(self, x, skip):
        x = self.up(x)
        # Ensure spatial dimensions match
        if x.shape[-2:] != skip.shape[-2:]:
            x = F.interpolate(x, size=skip.shape[-2:], mode='nearest')
        x = torch.cat((x, skip), dim=1)
        x = self.conv(x)
        return x

class EmbedFC(nn.Module):
    def __init__(self, input_dim, emb_dim):
        super(EmbedFC, self).__init__()
        '''
        This class defines a generic two-layer feed-forward neural network for embedding input data of
        dimensionality input_dim to an embedding space of dimensionality emb_dim.
        '''
        self.input_dim = input_dim

        # Define the layers for the network
        self.model = nn.Sequential(
            nn.Linear(input_dim, emb_dim),
            nn.GELU(),
            nn.Linear(emb_dim, emb_dim),
        )

    def forward(self, x):
        # Flatten the input tensor
        x = x.view(-1, self.input_dim)
        # Apply the model layers to the flattened tensor
        return self.model(x)

class MultiHeadSelfAttention(nn.Module):
    def __init__(self, dim, window_size=(16, 16), heads=8, shifted=False):
        super().__init__()
        self.dim = dim
        self.heads = heads
        self.window_size = window_size  # For relative positional encoding
        self.head_dim = dim // heads
        self.scale = self.head_dim ** -0.5
        self.shifted = shifted  # Whether to apply shifted window

        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)
        self.to_out = nn.Linear(dim, dim)

        # Relative positional bias table
        num_relative_distance = (2 * window_size[0] - 1) * (2 * window_size[1] - 1)
        self.relative_position_bias_table = nn.Parameter(
            torch.zeros(num_relative_distance, heads)
        )

        # Get pairwise relative position indices for each token in the window
        coords_h = torch.arange(window_size[0])
        coords_w = torch.arange(window_size[1])
        coords = torch.stack(torch.meshgrid(coords_h, coords_w))  # 2, Wh, Ww
        coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww

        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
        relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
        relative_coords[:, :, 0] += window_size[0] - 1  # Shift to start from 0
        relative_coords[:, :, 1] += window_size[1] - 1
        relative_coords[:, :, 0] *= 2 * window_size[1] - 1
        relative_position_index = relative_coords.sum(-1)  # Wh*Ww, Wh*Ww
        self.register_buffer("relative_position_index", relative_position_index)

        nn.init.trunc_normal_(self.relative_position_bias_table, std=0.02)

    def forward(self, x, mask=None):
        """
        x: (B, C, H, W)
        mask: (num_windows, num_windows) or None
        """
        B, C, H, W = x.shape
        window_h, window_w = self.window_size
        shifted = self.shifted

        if shifted:
            # Calculate shift size (usually half the window size)
            shift_size_h = window_h // 2
            shift_size_w = window_w // 2
            # Shift the feature map
            x = torch.roll(x, shifts=(-shift_size_h, -shift_size_w), dims=(2, 3))
        else:
            shift_size_h = 0
            shift_size_w = 0

        # Pad feature maps to multiples of window size
        pad_h = (window_h - H % window_h) if H % window_h != 0 else 0
        pad_w = (window_w - W % window_w) if W % window_w != 0 else 0
        x = F.pad(x, (0, pad_w, 0, pad_h))  # Pad last two dimensions
        B, C, H_padded, W_padded = x.shape

        # Create attention mask if shifted
        if shifted:
            # Calculate number of patches
            img_mask = torch.zeros((1, H_padded, W_padded, 1), device=x.device)
            # Compute the blocks
            h_blocks = H_padded // window_h
            w_blocks = W_padded // window_w

            # Create mask for shifted windows
            cnt = 0
            for h in range(h_blocks):
                for w in range(w_blocks):
                    img_mask[:, h * window_h:(h + 1) * window_h, w * window_w:(w + 1) * window_w, :] = cnt
                    cnt += 1

            mask_windows = self.window_partition(img_mask.permute(0, 3, 1, 2), window_h, window_w)  # (num_windows*B, window_h*window_w, 1)
            mask_windows = mask_windows.squeeze(-1)  # (num_windows*B, window_h*window_w)

            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        # Partition windows
        x_windows = self.window_partition(x, window_h, window_w)  # (num_windows*B, window_size*window_size, C)

        # Compute QKV
        qkv = self.to_qkv(x_windows).reshape(x_windows.shape[0], x_windows.shape[1], 3, self.heads, self.head_dim)
        q, k, v = qkv.permute(2, 0, 3, 1, 4).unbind(0)  # Each has shape (num_windows*B, heads, window_size*window_size, head_dim)

        # Compute attention
        attn = (q @ k.transpose(-2, -1)) * self.scale  # (num_windows*B, heads, window_size*window_size, window_size*window_size)

        # Add relative positional bias
        relative_position_bias = self.relative_position_bias_table[self.relative_position_index.view(-1)].view(
            window_h * window_w, window_h * window_w, -1
        )  # (window_size*window_size, window_size*window_size, heads)
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # (heads, window_size*window_size, window_size*window_size)
        attn = attn + relative_position_bias.unsqueeze(0)

        if shifted:
            # Apply attention mask
            attn = attn.view(B, -1, self.heads, window_h * window_w, window_h * window_w)
            attn = attn + attn_mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.heads, window_h * window_w, window_h * window_w)
            attn = attn.softmax(dim=-1)
        else:
            attn = attn.softmax(dim=-1)

        # Apply attention to V
        out = (attn @ v).transpose(1, 2).reshape(x_windows.shape[0], window_h * window_w, C)

        # Project out
        out = self.to_out(out)

        # Reverse window partitioning
        out = out.view(-1, window_h, window_w, C)
        x = self.window_reverse(out, H_padded, W_padded, window_h, window_w)  # (B, C, H_padded, W_padded)

        if shifted:
            # Reverse the shift
            x = torch.roll(x, shifts=(shift_size_h, shift_size_w), dims=(2, 3))

        # Remove padding
        if pad_h > 0 or pad_w > 0:
            x = x[:, :, :H, :W]

        return x

    def window_partition(self, x, window_h, window_w):
        """
        Args:
            x: (B, C, H, W)
            window_h, window_w: window size
        Returns:
            windows: (num_windows*B, window_size*window_size, C)
        """
        B, C, H, W = x.shape
        x = x.view(B, C, H // window_h, window_h, W // window_w, window_w)
        x = x.permute(0, 2, 4, 3, 5, 1).contiguous().view(-1, window_h * window_w, C)
        return x

    def window_reverse(self, windows, H, W, window_h, window_w):
        """
        Args:
            windows: (num_windows*B, window_size*window_size, C)
            H, W: Height and Width of image
            window_h, window_w: window size
        Returns:
            x: (B, C, H, W)
        """
        B = int(windows.shape[0] / (H * W / (window_h * window_w)))
        x = windows.view(B, H // window_h, W // window_w, window_h, window_w, -1)
        x = x.permute(0, 5, 1, 3, 2, 4).contiguous().view(B, -1, H, W)
        return x

class MultiHeadCrossAttention(nn.Module):
    def __init__(self, dim, context_dim, heads=1, dim_head=None):
        super().__init__()
        self.dim = dim
        self.context_dim = context_dim
        self.heads = heads
        self.dim_head = dim_head or (dim // heads)
        inner_dim = self.dim_head * heads
        self.scale = self.dim_head ** -0.5

        self.to_q = nn.Linear(dim, inner_dim, bias=False)
        self.to_k = nn.Linear(context_dim, inner_dim, bias=False)
        self.to_v = nn.Linear(context_dim, inner_dim, bias=False)
        self.to_out = nn.Linear(inner_dim, dim)

    def forward(self, x, context):
        b, c, h, w = x.shape

        x = x.view(b, c, -1).permute(0, 2, 1)  # Shape: [b, hw, c]
        q = self.to_q(x)
        k = self.to_k(context).unsqueeze(1)     # Shape: [b, 1, inner_dim]
        v = self.to_v(context).unsqueeze(1)     # Shape: [b, 1, inner_dim]

        q = q.view(b, -1, self.heads, self.dim_head).transpose(1, 2)  # [b, heads, hw, dim_head]
        k = k.view(b, -1, self.heads, self.dim_head).transpose(1, 2)  # [b, heads, 1, dim_head]
        v = v.view(b, -1, self.heads, self.dim_head).transpose(1, 2)  # [b, heads, 1, dim_head]

        attn = (q @ k.transpose(-2, -1)) * self.scale  # [b, heads, hw, 1]
        attn = attn.softmax(dim=-2)  # Softmax over the query dimension

        out = (attn @ v).transpose(1, 2).reshape(b, -1, self.dim)  # [b, hw, dim]
        out = self.to_out(out)

        out = out.permute(0, 2, 1).view(b, c, h, w)
        return out

class ContextUnet(nn.Module):
    def __init__(self, in_channels, n_feat=64, n_cfeat=1536):
        super(ContextUnet, self).__init__()
        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_cfeat = n_cfeat

        # Encoder
        self.init_conv = ResidualConvBlock(in_channels, n_feat, is_res=True)
        self.down1 = UnetDown(n_feat, 2 * n_feat)
        self.down2 = UnetDown(2 * n_feat, 4 * n_feat)
        self.down3 = UnetDown(4 * n_feat, 8 * n_feat)

        self.to_vec = nn.Identity()  # No spatial dimension reduction

        # Embeddings
        self.timeembed1 = EmbedFC(1, 8 * n_feat)
        self.timeembed2 = EmbedFC(1, 4 * n_feat)
        self.timeembed3 = EmbedFC(1, 2 * n_feat)

        self.contextembed1 = EmbedFC(n_cfeat, 8 * n_feat)
        self.contextembed2 = EmbedFC(n_cfeat, 4 * n_feat)
        self.contextembed3 = EmbedFC(n_cfeat, 2 * n_feat)

        # Decoder
        self.up0 = nn.Sequential(
            ResidualConvBlock(8 * n_feat, 8 * n_feat),
            ResidualConvBlock(8 * n_feat, 8 * n_feat),
        )
        self.up1 = UnetUp(in_channels=8 * n_feat, skip_channels=8 * n_feat, out_channels=4 * n_feat)
        self.up2 = UnetUp(in_channels=4 * n_feat, skip_channels=4 * n_feat, out_channels=2 * n_feat)
        self.up3 = UnetUp(in_channels=2 * n_feat, skip_channels=2 * n_feat, out_channels=n_feat)

        self.out = nn.Conv2d(n_feat, self.in_channels, 3, 1, 1)

        # Initialize attention layers with different window sizes and shifted flags
        self.self_attn1 = MultiHeadSelfAttention(dim=8 * n_feat, window_size=(8, 8), heads=8, shifted=False)
        self.self_attn2 = MultiHeadSelfAttention(dim=4 * n_feat, window_size=(4, 4), heads=8, shifted=True)
        self.self_attn3 = MultiHeadSelfAttention(dim=2 * n_feat, window_size=(2, 2), heads=8, shifted=False)

        self.cross_attn1 = MultiHeadCrossAttention(dim=8 * n_feat, context_dim=8 * n_feat, heads=1)
        self.cross_attn2 = MultiHeadCrossAttention(dim=4 * n_feat, context_dim=4 * n_feat, heads=1)
        self.cross_attn3 = MultiHeadCrossAttention(dim=2 * n_feat, context_dim=2 * n_feat, heads=1)

        # Positional Encoding
        self.pos_encoding = PositionalEncoding2D(n_feat)

        # Track shift state for alternating layers
        self.shift = False

    def forward(self, x, t, c=None):
        # Encoder
        x = self.init_conv(x)
        x = x + self.pos_encoding(x)  # Apply positional encoding
        down1, skip1 = self.down1(x)
        down2, skip2 = self.down2(down1)
        down3, skip3 = self.down3(down2)

        hiddenvec = self.to_vec(down3)

        if c is None:
            c = torch.zeros(x.shape[0], self.n_cfeat).to(x.device)

        c_embed1 = self.contextembed1(c).unsqueeze(-1).unsqueeze(-1)
        c_embed2 = self.contextembed2(c).unsqueeze(-1).unsqueeze(-1)
        c_embed3 = self.contextembed3(c).unsqueeze(-1).unsqueeze(-1)

        temb1 = self.timeembed1(t).unsqueeze(-1).unsqueeze(-1)
        temb2 = self.timeembed2(t).unsqueeze(-1).unsqueeze(-1)
        temb3 = self.timeembed3(t).unsqueeze(-1).unsqueeze(-1)

        # Decoder
        up1 = self.up0(hiddenvec)

        # Apply self-attention with window size (4x4)
        up1 = self.self_attn1(up1)
        up1 = self.cross_attn1(up1, c_embed1.squeeze(-1).squeeze(-1))
        up1 = up1 + temb1

        up1 = self.up1(up1, skip3)

        # Apply self-attention with window size (2x2)
        up2 = self.self_attn2(up1)
        up2 = self.cross_attn2(up2, c_embed2.squeeze(-1).squeeze(-1))
        up2 = up2 + temb2

        up2 = self.up2(up2, skip2)

        # Apply self-attention with window size (1x1)
        up3 = self.self_attn3(up2)
        up3 = self.cross_attn3(up3, c_embed3.squeeze(-1).squeeze(-1))
        up3 = up3 + temb3

        up3 = self.up3(up3, skip1)

        out = self.out(up3)
        return out

In [7]:
# hyperparameters

# diffusion hyperparameters
timesteps = 1000
beta1 = 1e-4
beta2 = 0.02




# network hyperparameters
# device = torch.device("cuda:0" if torch.cuda.is_available() else torch.device('cpu'))
n_feat = 128 # 128 hidden dimension feature
n_cfeat = 1536 # context vector is of size 1536
height = 128 # 128x128 image
save_dir = f"{system_directory}/Weights/"

# training hyperparameters
batch_size = 50
n_epoch = 200
lrate = 1e-5



n_gpus = torch.cuda.device_count()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
nn_model = ContextUnet(in_channels=3, n_feat=n_feat, n_cfeat=n_cfeat).to(device)
if n_gpus > 1:
    nn_model = nn.DataParallel(nn_model)

nn_model = nn_model.to(device)

/usr/local/lib/python3.11/dist-packages/torch/functional.py:534: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3595.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [8]:
# construct DDPM noise schedule
b_t = (beta2 - beta1) * torch.linspace(0, 1, timesteps + 1, device=device) + beta1
a_t = 1 - b_t
ab_t = torch.cumsum(a_t.log(), dim=0).exp()
ab_t[0] = 1

In [9]:
from torchinfo import summary

# Wrap the model
class ModelWrapper(nn.Module):
    def __init__(self, model):
        super(ModelWrapper, self).__init__()
        self.model = model

    def forward(self, x):
        if x.dim() == 3:
            x = x.unsqueeze(0)  # Add batch dimension if missing
        t = torch.zeros((x.size(0), 1), device=x.device)  # Dummy timestep
        c = torch.zeros((x.size(0), n_cfeat), device=x.device)  # Dummy context
        return self.model(x, t, c)
# nn_model = ContextUnet(in_channels=3, n_feat=n_feat, n_cfeat=n_cfeat).to(device)

model_wrapper = ModelWrapper(nn_model)

# Get the model summary
summary(model_wrapper, input_size=(1, 3, 128, 128))


Layer (type:depth-idx)                        Output Shape              Param #
ModelWrapper                                  [1, 3, 128, 128]          --
├─ContextUnet: 1-1                            [1, 3, 128, 128]          --
│    └─ResidualConvBlock: 2-1                 [1, 128, 128, 128]        --
│    │    └─GroupNorm: 3-1                    [1, 3, 128, 128]          6
│    │    └─GELU: 3-2                         [1, 3, 128, 128]          --
│    │    └─Conv2d: 3-3                       [1, 128, 128, 128]        3,584
│    │    └─GroupNorm: 3-4                    [1, 128, 128, 128]        256
│    │    └─GELU: 3-5                         [1, 128, 128, 128]        --
│    │    └─Conv2d: 3-6                       [1, 128, 128, 128]        147,584
│    │    └─Conv2d: 3-7                       [1, 128, 128, 128]        512
│    └─PositionalEncoding2D: 2-2              [1, 128, 128, 128]        --
│    └─UnetDown: 2-3                          [1, 256, 64, 64]          --
│    │    └

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader


transform = transforms.Compose([
    transforms.ToTensor(),                # from [0,255] to range [0.0,1.0]
    transforms.Normalize((0.5,), (0.5,))  # range [-1,1]
])
class SimpleDataset(Dataset):
    def __init__(self, data, transform=None):
        self.images = np.array([row[1] for row in data])[:, :, :, :3]
        self.embeddings = np.array([row[3] for row in data])
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        embedding = self.embeddings[idx]

        if self.transform:
            image = self.transform(image)

        return image, torch.tensor(embedding, dtype=torch.float32)

class CombinedDataset(Dataset):
    def __init__(self):
        self.data = []
        self.cumulative_sizes = [0]

    def add_dataset(self, dataset):
        self.data.extend([(dataset.images[i], dataset.embeddings[i]) for i in range(len(dataset))])
        self.cumulative_sizes.append(len(self.data))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image, embedding = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(embedding, dtype=torch.float32)

def create_combined_dataset(file_list, transform):
    combined_dataset = CombinedDataset()
    combined_dataset.transform = transform

    for file in file_list:
        data = np.load(file, allow_pickle=True)
        dataset = SimpleDataset(data, transform)
        combined_dataset.add_dataset(dataset)
        del dataset, data  # Remove references to free up memory

    return combined_dataset
# Usage example:
file_list = [
    f"{system_directory}/MetaData/gene_rgb_embedding.npy",


    # Add more dataset files as needed

]

dataset = create_combined_dataset(file_list, transform)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=4)



In [ ]:
# load 8 sample embeddings from dataset
sample_embeddings = torch.stack([dataset[i][1] for i in range(8)]).to(device)
print(sample_embeddings.shape)
print(sample_embeddings[0])

torch.Size([8, 1536])
tensor([-0.0304, -0.0100, -0.0075,  ..., -0.0166,  0.0087, -0.0098],
       device='cuda:0')


In [ ]:
# helper function; removes the predicted noise (but adds some noise back in to avoid collapse)
def denoise_add_noise(x, t, pred_noise, z=None):
    if z is None:
        z = torch.randn_like(x)
    noise = b_t.sqrt()[t] * z
    mean = (x - pred_noise * ((1 - a_t[t]) / (1 - ab_t[t]).sqrt())) / a_t[t].sqrt()
    return mean + noise

In [ ]:
@torch.no_grad()
def sample_ddpm_context(n_sample, context, save_rate=20):
    # x_T ~ N(0, 1), sample initial noise
    samples = torch.randn(n_sample, 3, height, height).to(device)

    # array to keep track of generated steps for plotting
    intermediate = []
    for i in range(timesteps, 0, -1):
        print(f'sampling timestep {i:3d}', end='\r')

        # reshape time tensor
        t = torch.tensor([i / timesteps])[:, None, None, None].repeat(n_sample, 1, 1, 1).to(device)

        # sample some random noise to inject back in. For i = 1, don't add back in noise
        z = torch.randn_like(samples) if i > 1 else 0

        # If using DataParallel, we need to handle the inputs differently
        if isinstance(nn_model, nn.DataParallel):
            # Prepare inputs
            inputs = (samples, t, context)

            # Run the model
            eps = nn_model(*inputs)
        else:
            eps = nn_model(samples, t, c=context)    # predict noise e_(x_t,t, ctx)

        samples = denoise_add_noise(samples, i, eps, z)
        if i % save_rate==0 or i==timesteps or i<8:
            intermediate.append(samples.detach().cpu().numpy())

    intermediate = np.stack(intermediate)
    return samples, intermediate

In [ ]:
# define sampling function for DDIM
# removes the noise using ddim
def denoise_ddim(x, t, t_prev, pred_noise):
    ab = ab_t[t]
    ab_prev = ab_t[t_prev]

    x0_pred = ab_prev.sqrt() / ab.sqrt() * (x - (1 - ab).sqrt() * pred_noise)
    dir_xt = (1 - ab_prev).sqrt() * pred_noise

    return x0_pred + dir_xt

In [ ]:
# fast sampling algorithm with context
@torch.no_grad()
def sample_ddim_context(n_sample, context, n=20):
    # x_T ~ N(0, 1), sample initial noise
    samples = torch.randn(n_sample, 3, height, height).to(device)

    # array to keep track of generated steps for plotting
    intermediate = []
    step_size = timesteps // n
    for i in range(timesteps, 0, -step_size):
        print(f'sampling timestep {i:3d}', end='\r')

        # reshape time tensor
        t = torch.tensor([i / timesteps])[:, None, None, None].to(device)

        eps = nn_model(samples, t, c=context)    # predict noise e_(x_t,t)
        samples = denoise_ddim(samples, i, i - step_size, eps)
        intermediate.append(samples.detach().cpu().numpy())

    intermediate = np.stack(intermediate)
    return samples, intermediate

In [ ]:
def visualize_embeddings_with_ep(sample_embeddings, ep):
    nn_model.load_state_dict(torch.load(f"{save_dir}/context_model_{ep}.pth", map_location=device))
    nn_model.eval()
    plt.clf()
    samples, intermediate = sample_ddpm_context(8, sample_embeddings)
    # samples, intermediate = sample_ddim_context(8, sample_embeddings)
    animation_ddpm_context = plot_sample(intermediate, 8, 2, save_dir, "ani_run", None, save=False)
    display(HTML(animation_ddpm_context.to_jshtml()))

In [ ]:
# helper function: perturbs an image to a specified noise level
def perturb_input(x, t, noise):
    return ab_t.sqrt()[t, None, None, None] * x + (1 - ab_t[t, None, None, None]) * noise

In [ ]:
# Training with context code
nn_model.train()

# Setup optimizer
optim = torch.optim.Adam(nn_model.parameters(), lr=lrate)

# Check if a checkpoint exists and load it if it does
checkpoint_path = f"{save_dir}checkpoint.pth"
start_epoch = 0
if os.path.exists(checkpoint_path):
    checkpoint = torch.load(checkpoint_path)
    nn_model.load_state_dict(checkpoint['model_state_dict'])
    optim.load_state_dict(checkpoint['optimizer_state_dict'])
    start_epoch = checkpoint['epoch'] + 1
    print(f"Resuming training from epoch {start_epoch}")

for ep in range(start_epoch, n_epoch):
    print(f'epoch {ep}')

    # linearly decay learning rate
    optim.param_groups[0]['lr'] = lrate*(1-ep/n_epoch)

    pbar = tqdm(dataloader, mininterval=2)
    epoch_loss = 0.0
    for x, c in pbar:   # x: images  c: context
        optim.zero_grad()
        x = x.to(device)
        c = c.to(x)

        # Randomly mask out c
        context_mask = torch.bernoulli(torch.zeros(c.shape[0]) + 0.9).to(device)
        c = c * context_mask.unsqueeze(-1)

        # Perturb data
        noise = torch.randn_like(x)
        t = torch.randint(1, timesteps + 1, (x.shape[0],)).to(device)
        x_pert = perturb_input(x, t, noise)

        # Use network to recover noise
        pred_noise = nn_model(x_pert, t / timesteps, c=c)

        # Loss is mean squared error between the predicted and true noise
        loss = F.mse_loss(pred_noise, noise)
        loss.backward()
        optim.step()

        # Accumulate the loss
        epoch_loss += loss.item()

    # Compute the average loss for the epoch
    avg_epoch_loss = epoch_loss / len(dataloader)
    print(f'Average loss for epoch {ep}: {avg_epoch_loss}')

    # Save model checkpoint
    if ep % 1 == 0 or ep == int(n_epoch - 1):
        if not os.path.exists(save_dir):
            os.mkdir(save_dir)

        # Update checkpoint
        torch.save({
            'epoch': ep,
            'model_state_dict': nn_model.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            'loss': avg_epoch_loss,
        }, checkpoint_path)
        print('saved checkpoint at ' + checkpoint_path)

        # Save model at ep
        torch.save(nn_model.state_dict(), save_dir + f"context_model_{ep}.pth")
        print('saved model at ' + save_dir + f"context_model_{ep}.pth")
        # Plot samples using current saved model
        visualize_embeddings_with_ep(sample_embeddings, ep)
        nn_model.train()  # Reset to training mode after evaluation
